# rrexpress - Extended Feature Selection
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Feb 22, 2022

## Hypothesis
Two very important components of a machine learning model are feature selection and feature engineering. Our idea is that adding some more features to the StellarAlgo retention model could improve performance of the model. This notebook will test the standard StellarAlgo retention model features.

## Experiment
This section details our experiment including querying data, data transformations, feature selection and modelling.

In [4]:
import getpass
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from pycaret.classification import *

warnings.filterwarnings('ignore')

### Let's connect to MSSQL and run a stored proc to get our dataset:

In [5]:
# connect to SQL Server.
SERVER = '34.206.73.189' 
DATABASE = 'datascience' 
USERNAME = 'nrad' 
PASSWORD = getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

In [6]:
lkupclientid = 34 # rrexpress

cursor = CNXN.cursor()

storedProc = (
    f"""Exec [stlrMILB].[ds].[getRetentionScoringModelData] {lkupclientid}"""
)

df = pd.read_sql(storedProc, CNXN)

# apply some data transformations
df["year"] = pd.to_numeric(df["year"])

CNXN.commit()
cursor.close()

df.head()
#df.info()

,lkupClientId,dimCustomerMasterId,customerNumber,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,source_tenure,tenure,distToVenue,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,click_link,fill_out_form,open_email,send_email,unsubscribe_email,openToSendRatio,clickToSendRatio,clickToOpenRatio,posting_records,resale_records,resale_atp,forward_records,cancel_records,email,inbound_email,inbound_phonecall,inperson_contact,internal_note,left_message,outbound_email,outbound_phonecall,phonecall,text,unknown,gender,childrenPresentInHH,maritalStatus,lengthOfResidenceInYrs,annualHHIncome,education,urbanicity,credits_after_refund,is_Lockdown,NumberofGamesPerSeason,CNTPostponedGames,isNextYear_Buyer
0,34,351611768,4207769,2017,Full Season,1278.00,2017-09-04,0.157143,136,TRUE,2190,287,62.24,11,10,2,0,7,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,1
1,34,351611946,5239536,2017,Fireworks Plans,806.96,2017-07-08,0.170455,37,TRUE,1825,186,7.61,5,8,0,1,2,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,0
2,34,351612032,4224129,2017,Full Season,639.00,1970-01-01,0.000000,105,TRUE,2190,256,22.76,0,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,1
3,34,351612167,4201956,2017,Full Season,1815.42,2017-09-04,0.658654,94,TRUE,2190,243,14.22,40,0,4,0,6,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,1
4,34,351612618,4203915,2017,Full Season,6390.00,2017-07-30,0.146429,58,TRUE,2190,242,31.30,14,17,0,0,7,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3637 entries, 0 to 3636
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lkupClientId            3637 non-null   int64  
 1   dimCustomerMasterId     3637 non-null   int64  
 2   customerNumber          3637 non-null   object 
 3   year                    3637 non-null   int64  
 4   productGrouping         3637 non-null   object 
 5   totalSpent              3637 non-null   float64
 6   recentDate              3637 non-null   object 
 7   attendancePercent       3637 non-null   float64
 8   renewedBeforeDays       3637 non-null   int64  
 9   isBuyer                 3637 non-null   object 
 10  source_tenure           3637 non-null   int64  
 11  tenure                  3637 non-null   int64  
 12  distToVenue             3637 non-null   float64
 13  totalGames              3637 non-null   int64  
 14  recency                 3637 non-null   

### Let's also drop the features that only have a single value, as they won't add much differentiation to our model:

In [8]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)
        
df.shape

(3637, 19)

### We should also drop features that have a low correlation with the target label as they won't be useful for prediction, we'll only keep features that have a correlation above a set threshold:

In [9]:
cor = df.corr()

threshold = 0.05

#Correlation with output variable
cor_target = abs(cor["isNextYear_Buyer"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target > threshold]

feats = []
for name, val in relevant_features.items():
    feats.append(name)

df_correlated = df[feats]

df_correlated.shape

df_correlated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3637 entries, 0 to 3636
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    3637 non-null   int64  
 1   totalSpent              3637 non-null   float64
 2   attendancePercent       3637 non-null   float64
 3   renewedBeforeDays       3637 non-null   int64  
 4   source_tenure           3637 non-null   int64  
 5   tenure                  3637 non-null   int64  
 6   distToVenue             3637 non-null   float64
 7   totalGames              3637 non-null   int64  
 8   missed_games_1          3637 non-null   int64  
 9   missed_games_2          3637 non-null   int64  
 10  missed_games_over_2     3637 non-null   int64  
 11  is_Lockdown             3637 non-null   int64  
 12  NumberofGamesPerSeason  3637 non-null   int64  
 13  isNextYear_Buyer        3637 non-null   int64  
dtypes: float64(3), int64(11)
memory usage: 3

### Now that we have the right features we can look at the correlations between them, if features are highly correlated with each other it might negatively impact the model:

In [10]:
corr = df_correlated.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,year,totalSpent,attendancePercent,renewedBeforeDays,source_tenure,tenure,distToVenue,totalGames,missed_games_1,missed_games_2,missed_games_over_2,is_Lockdown,NumberofGamesPerSeason,isNextYear_Buyer
year,1.00,0.01,-0.10,-0.23,-0.37,0.68,0.01,-0.10,-0.06,-0.08,-0.05,0.88,0.78,-0.22
totalSpent,0.01,1.00,-0.25,0.12,0.09,0.14,0.09,0.51,0.13,0.19,0.48,-0.01,0.02,0.24
attendancePercent,-0.10,-0.25,1.00,-0.10,-0.05,-0.20,0.04,0.09,0.11,-0.04,-0.45,-0.12,-0.05,-0.09
renewedBeforeDays,-0.23,0.12,-0.10,1.00,0.29,0.07,-0.06,0.37,0.28,0.26,0.27,-0.38,-0.05,0.41
source_tenure,-0.37,0.09,-0.05,0.29,1.00,0.23,-0.12,0.16,0.10,0.11,0.14,-0.34,-0.28,0.32
tenure,0.68,0.14,-0.20,0.07,0.23,1.00,-0.08,0.11,0.08,0.07,0.14,0.57,0.54,0.11
distToVenue,0.01,0.09,0.04,-0.06,-0.12,-0.08,1.00,0.04,0.02,0.03,-0.01,0.01,-0.00,-0.07
totalGames,-0.10,0.51,0.09,0.37,0.16,0.11,0.04,1.00,0.63,0.53,0.47,-0.14,-0.01,0.43
missed_games_1,-0.06,0.13,0.11,0.28,0.10,0.08,0.02,0.63,1.00,0.54,0.24,-0.10,0.02,0.26
missed_games_2,-0.08,0.19,-0.04,0.26,0.11,0.07,0.03,0.53,0.54,1.00,0.37,-0.10,-0.02,0.25


### In order to compare two sets of features, we need to create some datasets for training and evalution:


In [11]:
# select % of the data for training
df_train = df_correlated.sample(frac=0.8, random_state=786).reset_index(drop=True)

# create the eval datasets for A and B
df_eval = df_correlated.drop(df_train.index).reset_index(drop=True)

# print out the number of records for training and eval
print('Data for Modeling: ' + str(df_train.shape))
print('Unseen Data For Predictions: ' + str(df_eval.shape), end="\n\n")

Data for Modeling: (2910, 14)
Unseen Data For Predictions: (727, 14)



### Now we can model the data using a binary classification prediction for the isnextyear_buyer field to see how likely a customer is to re-purchase.

In [12]:
setup(
    data= df_train, 
    target="isNextYear_Buyer", 
    train_size = 0.80,
    data_split_shuffle=True,
    silent=True,
    numeric_features=[
        "totalSpent",
        "attendancePercent",
        "renewedBeforeDays",
        "source_tenure",
        "tenure",
        "distToVenue",
        "totalGames",
        "missed_games_1",
        "missed_games_2",
        "missed_games_over_2",
        "is_Lockdown",
        "NumberofGamesPerSeason"
    ]
)

,Description,Value
0,session_id,6220
1,Target,isNextYear_Buyer
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(2910, 14)"
5,Missing Values,False
6,Numeric Features,12
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


(       totalSpent  attendancePercent  renewedBeforeDays  source_tenure  \
 1466  1260.000000           0.330435              190.0         2190.0   
 1943  2660.000000           0.315217              126.0         1460.0   
 1268   750.000000           0.985507              210.0         2190.0   
 1336   880.000000           0.500000               63.0         2190.0   
 1499   263.500000           0.083333               43.0         2190.0   
 ...           ...                ...                ...            ...   
 2239   125.000000           1.000000                3.0         1460.0   
 516     77.940002           0.833333               29.0         2190.0   
 2549   792.000000           0.069444               43.0         2190.0   
 628    140.699997           1.000000               15.0         2190.0   
 577    115.500000           1.000000                0.0         1095.0   
 
       tenure  distToVenue  totalGames  missed_games_1  missed_games_2  \
 1466  1004.0     3.9200

In [13]:
model_matrix = compare_models(
    fold=10,
    include=["lr", "xgboost"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7702,0.8411,0.8190,0.7675,0.7919,0.5359,0.5382,0.3870
lr,Logistic Regression,0.7698,0.8505,0.7876,0.7847,0.7851,0.5371,0.5386,0.4240


In [14]:
best_model = create_model(model_matrix)
final_model = finalize_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7725,0.8533,0.8320,0.7647,0.7969,0.5394,0.5419
1,0.7468,0.8316,0.7600,0.7661,0.7631,0.4912,0.4912
2,0.7682,0.8244,0.8240,0.7630,0.7923,0.5310,0.5331
3,0.7854,0.8605,0.8548,0.7681,0.8092,0.5657,0.5699
4,0.8069,0.8739,0.8065,0.8264,0.8163,0.6128,0.6130
5,0.8326,0.8804,0.9032,0.8058,0.8517,0.6610,0.6667
6,0.6953,0.7410,0.7823,0.6879,0.7321,0.3822,0.3864
7,0.7983,0.8668,0.8548,0.7852,0.8185,0.5924,0.5951
8,0.7672,0.8633,0.7823,0.7823,0.7823,0.5323,0.5323
9,0.7284,0.8153,0.7903,0.7259,0.7568,0.4507,0.4528


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

## Observations
Here you can document some ideas on the results from above

## Conclusions
Here you can talk about next steps, did the experiment work? If yes, what to do next? If no, why?